# CAPM model simplified

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

## Download history data

In [ ]:
# Download stock 
tickers = ["SPY", "AAPL", "META", "NVDA", "AMZN"]

from_time = "2023-01-01"
to_time = "2024-06-15"

historical_data = yf.download(tickers, from_time, to_time)

historical_data

In [ ]:
# Plotting Adj Close prices of the stock with legend for each stock
plt.figure(figsize=(12, 8))

for ticker in tickers:
    plt.plot(historical_data['Close'][ticker], label=ticker)

# Adding title and labels
plt.title('Close Prices of Major Tech Stocks')
plt.ylabel('Stock Price')
plt.xlabel('Date')
plt.grid(True)
plt.legend()

plt.show()

In [ ]:
# Calculate mean, median, mode, standard deviation, variance, skewness, and kurtosis
stats_metrics = historical_data['Close'].describe().T

# display the statistics metrics
display(stats_metrics)

# show Box Plot of Close prices of the stock ( show Mean, Median, 1st Quartile, 3rd Quartile, Max, Min, Outliers)
plt.figure(figsize=(12, 8))
historical_data['Close'].plot(kind='box', figsize=(15, 10))
plt.title('Box Plot of Adj Close Prices')
plt.ylabel('Stock Price')
plt.xlabel('Ticker')

plt.show()

In [ ]:
log_returns = np.log(historical_data['Close'] / historical_data['Close'].shift(1))

# drop NaN values
log_returns.dropna(inplace=True)

# plot Log Returns of the stock
plt.figure(figsize=(12, 8))
log_returns.plot(figsize=(15, 10))
plt.title('Log Returns of Major Tech Stocks')
plt.ylabel('Log Returns')
plt.xlabel('Date')
plt.grid(True)
plt.show()

### Calculate
- expected return
- covariance
- variance
- beta coefficient
- jensens alpha

In [ ]:
# risk free rate, for now 4% per annum
risk_free_rate = 0.0138

# calculate annualized mean returns
mean_returns = log_returns.mean() * 252

# calculate covariance matrix
cov_matrix = log_returns.cov()

# calculate annualized std
std = log_returns.std() * np.sqrt(252)

# calculate beta coefficient = cov / std of SPY
beta_coefficients = cov_matrix["SPY"].div(log_returns["SPY"].var(), axis=0)

# calculate expected returns using CAPM
expected_returns = risk_free_rate + beta_coefficients * (mean_returns["SPY"] - risk_free_rate)

# calculate jensen's alpha, the abnormal return of the stock
jensen_alpha = mean_returns - expected_returns

In [ ]:
# print Risk-Free Rate, Mean returns of SPY
print('-'*50)
print('Risk-Free Rate:', risk_free_rate)
print('-'*50)
print('Mean Returns of SPY:', mean_returns['SPY'])

# display expected returns, covariance matrix, variance, and beta coefficients
print('-'*50)
print('Covariance Matrix:')
display(cov_matrix)
print('-'*50)
print('Standard Deviation:')
display(std)
print('-'*50)
print('Beta Coefficients:')
display(beta_coefficients)
print('-'*50)

# put expected returns & mean returns & Beta & Jensen's Alpha in a DataFrame
df_returns = pd.DataFrame([expected_returns, mean_returns, beta_coefficients, jensen_alpha], index=['Expected Returns', 'Mean Returns', 'Beta', "Jensen's Alpha"]).T
display(df_returns)

# plot beta coefficients ( remove SPY)
plt.figure(figsize=(12, 8))
df_returns['Beta'].drop('SPY').plot(kind='bar', figsize=(15, 10))
plt.title('Beta Coefficients of Major Tech Stocks')
plt.ylabel('Beta Coefficient')
plt.xlabel('Ticker')
plt.grid(True)

# plot Jensen's Alpha ( remove SPY)
plt.figure(figsize=(12, 8))
df_returns["Jensen's Alpha"].drop('SPY').plot(kind='bar', figsize=(15, 10))
plt.title('Jensen\'s Alpha of Major Tech Stocks')
plt.ylabel('Jensen\'s Alpha')
plt.xlabel('Ticker')
plt.grid(True)

# Efficient Frontier of portfolios (no risk free rate)
goal:
- max sharpe ratio
- min volatility
- min VaR

In [ ]:
# num of simulations
num_simulations = 10000
risk_free_rate = 0.0138
confidence_level = 0.05 # 95% confidence level

# stock returns
stock_returns = log_returns.drop("SPY",axis=1)
stock_tickers = stock_returns.columns

# annualized mean returns
mean_returns = stock_returns.mean() * 252
# annualized covariance matrix
cov_matrix = stock_returns.cov() * 252

# calculate portfolio performance metrics
def calc_portfolio_perf(weights, mean_returns, cov_matrix, rf):
    # compute portfolio log return annualized
    portfolio_return = np.sum(mean_returns * weights)
    # compute portfolio std annualized
    portfolio_std = np.sqrt(weights.T @ cov_matrix @ weights)
    # compute sharpe ratio
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    # compute portfolio Value at Risk (VaR) using the normal distribution
    portfolio_var = norm.ppf(1 - confidence_level, portfolio_return, portfolio_std)
    return portfolio_return, portfolio_std, sharpe_ratio, portfolio_var

# prepare monte carlo simulation
results = np.zeros((len(stock_tickers), num_simulations))
weights_record = []

# randomize weights and calculate portfolio performance metrics
for i in range(num_simulations):
    weights = np.random.random(len(stock_tickers))
    weights /= np.sum(weights)
    weights_record.append(weights)
    portfolio_return, portfolio_std, sharpe_ratio, portfolio_var = calc_portfolio_perf(weights, mean_returns, cov_matrix, risk_free_rate)
    results[0, i] = portfolio_return
    results[1, i] = portfolio_std
    results[2, i] = sharpe_ratio
    results[3, i] = portfolio_var

results[4] = np.arange(num_simulations)

# get the indices of the optimal portfolios
max_sharpe_idx = np.argmax(results[2])
min_vol_idx = np.argmin(results[1])
min_var_idx = np.argmin(results[3])

# get portfolio metrics for the optimal portfolios
max_sharpe_port = results[:, max_sharpe_idx]
min_vol_port = results[:, min_vol_idx]
min_var_port = results[:, min_var_idx]

# get the weights for the optimal portfolios using simulation index
max_sharpe_weights = weights_record[int(max_sharpe_port[4])]
min_vol_weights = weights_record[int(min_vol_port[4])]
min_var_weights = weights_record[int(min_var_port[4])]


In [ ]:
# Visualize the efficient frontiers
plt.figure(figsize=(10, 6))
plt.scatter(results[1, :], results[0, :], c=results[2, :], cmap='RdYlBu', marker='o')
plt.colorbar(label='Sharpe Ratio')
plt.scatter(max_sharpe_port[1], max_sharpe_port[0], marker='*', color='r', s=500, label='Max Sharpe Ratio')
plt.scatter(min_vol_port[1], min_vol_port[0], marker='*', color='g', s=500, label='Min Volatility')
plt.scatter(min_var_port[1], min_var_port[0], marker='*', color='b', s=500, label='Min VaR')
plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
plt.xlabel('Standard Deviation')
plt.ylabel('Returns')
plt.legend(labelspacing=0.8)
plt.show()

print("Max Sharpe Ratio Portfolio Allocation\n")
print("Annualised Return:", round(max_sharpe_port[0], 2))
print("Annualised Volatility:", round(max_sharpe_port[1], 2))
print("Sharpe Ratio:", round(max_sharpe_port[2], 2))
print("Weights:", dict(zip(stock_tickers, max_sharpe_weights)))
print("\n")

print("Min Volatility Portfolio Allocation\n")
print("Annualised Return:", round(min_vol_port[0], 2))
print("Annualised Volatility:", round(min_vol_port[1], 2))
print("Sharpe Ratio:", round(min_vol_port[2], 2))
print("Weights:", dict(zip(stock_tickers, min_vol_weights)))
print("\n")

print("Min VaR Portfolio Allocation\n")
print("Annualised Return:", round(min_var_port[0], 2))
print("Annualised Volatility:", round(min_var_port[1], 2))
print("Sharpe Ratio:", round(min_var_port[2], 2))
print("Weights:", dict(zip(stock_tickers, min_var_weights)))